#### Term Use Cases

The SAS Glossary API allows users to create, read, update, and delete terms. The SAS Catalog API can be used to search for terms and to associate the terms to columns. The provided use cases will demonstrate how to accomplish each of these tasks.

##### **Imports and Global Variables** <a id='imports'></a>
Run this cell before any of the others. It imports packages and sets variables that will be used throughout the notebook.

In [ ]:
import requests
import json
sasserver = "https://your_server"
print (sasserver)
prefix = "Sample"

##### Get Access token <a id='authentication'></a>
This use case sets the access_token that will be used in all  the calls in this use case. Edit the payload object with a username and password. Run this use case as needed because the token will expire after a certain amount of time.

In [ ]:
url = sasserver + "/SASLogon/oauth/token#password"

payload={
    'grant_type': 'password',
    'username': 'your_username',
    'password': 'your_password'
}
files=[

]
headers = {
  'Authorization': 'Basic c2FzLmVjOg==',
}

response = requests.request("POST", url, headers=headers, data=payload, files=files, verify=False)

if response.status_code == 200:
    print("Successfully authenticated.")
    response = response.json()
    access_token = response["access_token"]
else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

##### Create a Term Type
This use case creates a term type that will be used in the following examples. Specify an appropriate name, label, and description. You also can specify custom attributes, if they are necessary.

In [ ]:

url = sasserver + "/glossary/termTypes"


payload = json.dumps({
  "name": prefix + " Term Type",
  "description": "Sample Term Type with all attribute values populated.",
  "label": "Sample Term Type",
  "attributes": [
    {
      "name": "singleLineTextAttribute",
      "label": "A single-line text attribute",
      "description": "This is a description of the attribute. Single-line text attributes can be up to 4000 characters long and cannot contain newline characters.",
      "required": True,
      "type": "single-line",
      "defaultValue": "This is an example of a single-line text attribute."
    },
    {
      "name": "multiLineTextAttribute",
      "label": "A multi-line text attribute",
      "description": "Multi-line text attributes can be up to 4000 characters.  Newline characters are allowed.",
      "type": "multi-line",
      "defaultValue": "This is an example of a multi-line text attribute\\nwith\\nnewline\\ncharacters."
    },
    {
      "name": "booleanAttribute",
      "label": "A boolean attribute",
      "description": "Boolean attributes can be either \"true\" or \"false\".",
      "type": "boolean",
      "defaultValue": "true"
    },
    {
      "name": "dateAttribute",
      "label": "A date attribute",
      "description": "Date attributes should be specified with the format \"YYYY-MM-DD\".",
      "type": "date"
    },
    {
      "name": "timeAttribute",
      "label": "A time attribute",
      "description": "Time attributes should be specified with the format \"hh:mm:ssZ\".",
      "type": "time",
      "defaultValue": "10:23:45Z"
    },
    {
      "name": "dateTimeAttribute",
      "label": "A date-time attribute",
      "description": "Date-time attributes should be specified with the format \"YYYY-MM-DDThh:mm:ssZ\".",
      "type": "date-time",
      "defaultValue": "2024-02-12T10:23:45.000Z"
    },
    {
      "name": "singleSelectAttribute",
      "label": "A single-select attribute",
      "description": "Single-select attributes must specify at least one item in the \"items\" field.",
      "type": "single-select",
      "items": [
        "Item 1",
        "Item 2",
        "Item 3"
      ],
      "defaultValue": "Item 1"
    }
  ]
})

headers = {
  'Content-Type': 'application/vnd.sas.glossary.term.type+json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

if response.status_code == 201:
    print(f"Successfully created term type.\n{json.dumps(response.json(), indent=2)}")
    response = response.json()
    termTypeId = response["id"]
else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

##### Create a Term
This use case creates a term. This term will act as the parent term.

In [ ]:
url = sasserver + "/glossary/terms?publish=true"

payload = json.dumps({
  "name": prefix + " Term",
  "description": "Sample Term With Attributes",
  "label": "Sample Term",
  "termTypeId": termTypeId,
  "attributes": {
    "singleLineTextAttribute": "This is an example of a single-line text attribute.",
    "multiLineTextAttribute": "This is an example of a multi-line text attribute\\nwith\\nnewline\\ncharacters.",
    "singleSelectAttribute": "Item 2",
    "booleanAttribute": True,
    "dateAttribute": "2024-05-24",
    "timeAttribute": "09:50:59Z",
    "dateTimeAttribute": "2024-05-24T09:50:59Z"
  }
})
headers = {
  'Accept': 'application/vnd.sas.glossary.term+json',
  'Content-Type': 'application/vnd.sas.glossary.term+json',
  'Authorization': 'Bearer ' + access_token}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

if response.status_code == 201:
    print("Successfully created term.")
    parentEtag = response.headers["ETag"]
    response = response.json()
    print(json.dumps(response, indent=2))
    parentId = response["id"]

else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")



##### Create a Child Term
This use case creates a term that is a child of another term.

In [ ]:
url = sasserver + "/glossary/terms?publish=true"

payload = json.dumps({
  "name": prefix + " Child Term",
  "description": "Sample Child Term With Atributes",
  "label": "Sample Child Term",
  "termTypeId": termTypeId,
  "parentId": parentId,
  "attributes": {
    "singleLineTextAttribute": "This is an example of a single-line text attribute.",
    "multiLineTextAttribute": "This is an example of a multi-line text attribute\\nwith\\nnewline\\ncharacters.",
    "singleSelectAttribute": "Item 3",
    "booleanAttribute": True,
    "dateAttribute": "2024-05-24",
    "timeAttribute": "09:50:59Z",
    "dateTimeAttribute": "2024-05-24T09:50:59Z"
  }
})
headers = {
  'Accept': 'application/vnd.sas.glossary.term+json',
  'Content-Type': 'application/vnd.sas.glossary.term+json',
  'Authorization': 'Bearer ' + access_token}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

if response.status_code == 201:
    print("Successfully created term.")
    childEtag = response.headers["ETag"]
    response = response.json()
    print(json.dumps(response, indent=2))
    childId = response["id"]
else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

##### Get a Term by Id
This use case retrieves a term with a specified ID.

In [ ]:
url = sasserver + "/glossary/terms/" + parentId +"?allowDrafts=none"

payload = {}
headers = {
  'Accept': 'application/vnd.sas.glossary.term+json',
  'Authorization': 'Bearer ' + access_token}

response = requests.request("GET", url, headers=headers, data=payload, verify=False)

if response.status_code == 200:
    print("Successfully retrieved term.")
    response = response.json()
    print(json.dumps(response, indent=2))
else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")


##### Get Terms by Filter
This use case retrieves all terms that meet the specified filter criteria.

In [ ]:
url = sasserver + "/glossary/terms?filter=startsWith(name," + prefix + ")&allowDrafts=none"

payload = {}
headers = {
  'Accept': 'application/vnd.sas.collection+json',
  'Authorization': 'Bearer ' + access_token}

response = requests.request("GET", url, headers=headers, data=payload, verify=False)

if response.status_code == 200:
    print("Successfully retrieved terms.")
    response = response.json()
    print(json.dumps(response, indent=2))
else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

##### Update a Term
This use case updates an existing term by ID with the specified payload.

In [ ]:
url = sasserver + "/glossary/terms/" + parentId

payload = json.dumps({
  "id": parentId,
  "name": prefix + " Updated Term",
  "description": "Sample Updated Term",
  "label": "Sample Updated Term",
  "termTypeId": termTypeId,
  "attributes": {
    "singleLineTextAttribute": "This is an example of a single-line text attribute.",
    "multiLineTextAttribute": "This is an example of a multi-line text attribute\\nwith\\nnewline\\ncharacters.",
    "singleSelectAttribute": "Item 1",
    "booleanAttribute": True,
    "dateAttribute": "2024-05-24",
    "timeAttribute": "09:50:59Z",
    "dateTimeAttribute": "2024-05-24T09:50:59Z"
  }
})
headers = {
  'Accept': 'application/vnd.sas.glossary.term+json',
  'Content-Type': 'application/vnd.sas.glossary.term+json',
  'If-Match': parentEtag,
  'Authorization': 'Bearer ' + access_token}

response = requests.request("PUT", url, headers=headers, data=payload, verify=False)

if response.status_code == 200:
    print("Successfully updated term.")
    response = response.json()
    print(json.dumps(response, indent=2))
else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

#### Patch a Term
This use case changes the Parent ID of a Term using patch op: add method. 
This use case removes the parentId. 

In [ ]:
url = sasserver + "/glossary/terms/" + childId

payload = json.dumps([
  {
    "op": "add",
    "path": "/parentId",
    "value": ""
  }
])
headers = {
  'Content-Type': 'application/json-patch+json',
  'Accept': 'application/vnd.sas.glossary.term+json',
  'If-Match': childEtag,
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("PATCH", url, headers=headers, data=payload, verify=False)

if response.status_code == 200:
    childEtag = response.headers["ETag"]
    response = response.json()
    print("Successfully patched term.")
    print(json.dumps(response, indent=2))

else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

##### Search Terms
This use case demonstrates how to use the search API to find terms.

In [ ]:
url = sasserver +  "/catalog/search?q=*&indices=terms"

payload = {}
headers = {
  'Accept': 'application/json',
  'Accept-Item': 'application/vnd.sas.metadata.search.result+json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("GET", url, headers=headers, data=payload, verify=False)

if response.status_code == 200:
    print("Successfully retrieved terms.")
    response = response.json()
    print(json.dumps(response, indent=2))
else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

##### Create Table Entity Instance
This use case creates a table instance that is used later in the "Assigning a Term to a Column" example.

In [ ]:
url = sasserver + "/catalog/instances"

payload = json.dumps({
  "name": prefix +" Table",
  "type": "casTable",
  "instanceType": "entity",
  "definition": "casTable",
  "resourceId": "/dataTables/dataSources/cas~fs~cas-shared-default~fs~Public/tables/ATable"
})
headers = {
  'Accept': 'application/vnd.sas.metadata.instance.entity+json',
  'Content-Type': 'application/json',
  'Authorization': 'Bearer ' + access_token}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

if response.status_code == 201:
    print("Successfully created instance.")
    response = response.json()
    print(json.dumps(response, indent=2))
    tableId = response["id"]

else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

##### Create Column Entity Instance
This use case creates a column instance that is used later in the "Assigning a Term to a Column" example.

In [ ]:
url = sasserver + "/catalog/instances"

payload = json.dumps({
  "instanceType": "entity",
  "definition": "casColumn",
  "definitionId": "87d48d12-53c0-473c-ab34-531817319e79",
  "name": prefix + " Column",
  "type": "casColumn",
  "attributes": {
    "analysisTimeStamp": "2023-09-05T08:55:23.435Z",
    "max": 2975
  },
  "resourceId": "/dataTables/dataSources/cas~fs~cas-shared-default~fs~Public/tables/Table/columns/Column"
})
headers = {
  'Accept': 'application/vnd.sas.metadata.instance.entity+json',
  'Content-Type': 'application/json',
  'Authorization': 'Bearer ' + access_token}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

if response.status_code == 201:
    print("Successfully created instance.")
    response = response.json()
    print(json.dumps(response, indent=2))
    columnId = response["id"]

else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

##### Create Table-Column Relationship
This use case establishes the relationship between the table and column above.

In [ ]:
url = sasserver + "/catalog/instances"

payload = json.dumps({
  "name": prefix + " Column And Table Relationship",
  "label": "Column And Table Relationship",
  "description": "Column And Table Relationship",
  "instanceType": "relationship",
  "endpoint1Id": tableId,
  "endpoint2Id": columnId,
  "definition": "dataSetDataFields"
})
headers = {
  'Accept': 'application/vnd.sas.metadata.instance.relationship+json',
  'Content-Type': 'application/vnd.sas.metadata.instance.relationship+json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

if response.status_code == 201:
    print("Successfully created relationship.")
    response = response.json()
    print(json.dumps(response, indent=2))
    tableColumnId = response["id"]

else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

##### Assigning a Term to a Column
This use case creates a termAsset relationship that assigns a term to a dataset column.

In [ ]:
url = sasserver + "/catalog/instances"

payload = json.dumps({
  "name": prefix + " Term And Column Relationship",
  "label": "Term And Column Relationship",
  "description": "Term And Column Relationship",
  "instanceType": "relationship",
  "endpoint1Id": parentId,
  "endpoint2Id": columnId,
  "definition": "termAsset"
})
headers = {
  'Accept': 'application/vnd.sas.metadata.instance.relationship+json',
  'Content-Type': 'application/vnd.sas.metadata.instance.relationship+json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

if response.status_code == 201:
    print("Successfully created relationship.")
    response = response.json()
    print(json.dumps(response, indent=2))
    termAssetId = response["id"]
else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

##### Search For a Term
This use case demonstrates using Catalog search to find a term.

In [ ]:
url = sasserver + "/catalog/search?q=Column.Term:\"" + prefix + " Term\""

payload = {}
headers = {
  'Accept': 'application/json',
  'Accept-Item': 'application/vnd.sas.metadata.search.result+json',
  'Authorization': 'Bearer ' + access_token}

response = requests.request("GET", url, headers=headers, data=payload, verify=False)

if response.status_code == 200:
    print("Successfully retrieved asset.")
    response = response.json()
    print(json.dumps(response, indent=2))
else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

##### Get Terms Assigned to a Column
This use case demonstrates retrieving the termAsset relationships that associate terms with a specific column.

In [ ]:
url = sasserver + "/catalog/instances?limit=10&filter=contains(name, \"" + prefix + " Term And Column Relationship\")"

payload = {}
headers = {
  'Accept-Item': 'application/vnd.sas.metadata.instance.relationship+json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("GET", url, headers=headers, data=payload, verify=False)

if response.status_code == 200:
    print("Successfully retrieved terms.")
    response = response.json()
    print(json.dumps(response, indent=2))
else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

##### Disassociate a Term from a Column
This use case deletes the relationship between a term and a column.

In [ ]:
url = sasserver + "/catalog/instances/" + termAssetId

payload = {}
headers = {
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("DELETE", url, headers=headers, data=payload, verify=False)

if response.status_code == 204:
    print("Successfully deleted relationship.")

else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

##### Delete a Term
This use case deletes the specified term by ID.

In [ ]:
url = sasserver + "/glossary/terms/" + childId

headers = {
     'Authorization': 'Bearer ' + access_token
}

response = requests.request("DELETE", url, headers=headers, data=payload, verify=False)

if response.status_code == 204:
    print("Successfully deleted term.")

else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")

##### Clean up
This use case deletes terms and term types that were created by these examples.

In [ ]:
# Getting All Terms
url = sasserver + "/catalog/instances?filter=startsWith(name," + prefix + ")&limit=20&sortBy=creationTimeStamp:descending"

headers = {
  'Accept': 'application/vnd.sas.collection+json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("GET", url, headers=headers, data=payload, verify=False)

if response.status_code == 200:
    instances = response.json().get('items', [])

    host = sasserver
    token = access_token

    # Deleting Terms
    for index, a_term in enumerate(instances):
        del_link = next(link['href'] for link in a_term['links'] if link['rel'] == 'delete')

        del_request_url = host + del_link
        del_request_headers = {'Authorization': 'Bearer ' + token}

        response = requests.delete(del_request_url, headers=del_request_headers, verify=False)
        if response.status_code == 204:
            print('Deleted term with id:', a_term['id'])
        else:
            print('Error deleting term:', response.text)

    print('All terms have been processed.')

else:
    print(f"Error retrieving terms:\n{json.dumps(response.json(), indent=2)}")

# Deleting Term Types
url = sasserver + "/glossary/termTypes/" + termTypeId

payload = {}
headers = {
  'Accept': 'application/vnd.sas.glossary.term.type+json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("DELETE", url, headers=headers, data=payload, verify=False)

if response.status_code == 204:
    print("Successfully deleted term type.")

else:
    print(
        f"Error. Here is the response:\n{json.dumps(response.json(), indent=2)}")